In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

from utils import *

device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

In [2]:
input_lang, output_lang, pairs = prepareData('spa', 'eng', True)
print(random.choice(pairs))

Reading lines...
Read 128084 sentence pairs
Trimmed to 7961 sentence pairs
Counting words...
Counted words:
eng 3958
spa 2730
['estoy acostumbrada al tiempo frio .', 'i am accustomed to cold weather .']


In [3]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)